In [41]:
import smtplib, ssl
import pandas as pd
import requests
from bs4 import BeautifulSoup
from price_parser import Price
import re

PRODUCT_URL_CSV = "products.csv"
SAVE_TO_CSV = True
PRICES_CSV = "prices.csv"
SEND_MAIL = True

def addProduct(csv_file):
    df = readUrls(csv_file)
    productName = input("Enter Product Name: ")
    productUrl = input("Enter Product URL: ")
    alertPrice = float(input (" Enter Alert Price to set: "))
    #Method 1 using append
    #new_row = pd.Series({'product': productName, 'url': productUrl,'alertPrice': alertPrice})
    #df = df.append(new_row, ignore_index=True)
   
    #Method 2 w/o append as it is deprecated
    new_row = pd.Series({'product': productName, 'url': productUrl,'alertPrice': alertPrice},name=len(df))
    df.loc[len(df)] = new_row
    df.to_csv(csv_file)
    return df

def readUrls(csv_file):
    df = pd.read_csv(csv_file)    
    return df

def getResponse(url):
    headers = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    response = requests.get(url,headers = headers)
    return response.text

def getPrice(html):

    soup = BeautifulSoup(html, "html.parser")
    #alternate way to locate price in AZ
    #el ="₹1,21,999" #soup.find('span', {'class' :'a-price-whole'} ) #a-price-whole"  /a-offscreen
    priceEl = soup.find("span", class_="a-offscreen")
    #priceEl = soup.find("span", class_="a-price-whole")  doesn't work for all products
    if priceEl:
        price = priceEl.get_text()
        priceNew = price.replace(",", "").replace("₹","")
    else:
        priceNew = 0
    return float(priceNew)


def formatProducts(df):
    updated_products = []
    for product in df.to_dict("records"):
        html = getResponse(product["url"])
        product["price"] = getPrice(html) 
        print(product["price"]) #price check
        product["alert"] = product["price"] <= product["alertPrice"]
        updated_products.append(product)
    print(updated_products)
    return pd.DataFrame(updated_products)

def getMail(df):
    subject = "Price Drop Alert"
    body = df[df["alert"]].to_string()
    subject_and_message = f"Subject:{subject}\n\n{body}"
    return subject_and_message

def send_mail(df):
    messageText = getMail(df)
    smtpServer = "smtp.gmail.com"
    senderEmail =" # enter sender email"
    password = "#Enter password of sender email"
    receiverEmail = #enter receiver email"
    context = ssl.create_default_context()
    with smtplib.SMTP(smtpServer, 587) as smtp:
        try:
            smtp.starttls(context = context)
            smtp.login(senderEmail,password)
            smtp.sendmail(senderEmail,receiverEmail, messageText)
        except Exception as e:
            print(e)

def mainDriver():
    df = readUrls(PRODUCT_URL_CSV)
    display(df)
    ch = input(" Would you like to add a product? ")
    if(ch == 'Yes'):
        df = addProduct(PRODUCT_URL_CSV)
        print("added")
    display(df)
    df_updated = formatProducts(df)
    if SAVE_TO_CSV:
        df_updated.to_csv(PRICES_CSV, index=False, mode="a")
        print("running")
    if SEND_MAIL:
      send_mail(df_updated)
      print("mailed")


In [42]:
mainDriver()

,product,url,alertPrice
0,Nothing Phone(1),https://www.amazon.in/Nothing-Phone-Black-128-...,27950
1,iPhone 14,https://www.amazon.in/Apple-iPhone-128GB-Space...,121990


added


,product,url,alertPrice
0,Nothing Phone(1),https://www.amazon.in/Nothing-Phone-Black-128-...,27950.0
1,iPhone 14,https://www.amazon.in/Apple-iPhone-128GB-Space...,121990.0
2,Nike Court Shoes,https://www.amazon.in/Nike-Legacy-Sail-Vivid-G...,3200.0


26999.0
122999.0
3124.0
[{'product': 'Nothing Phone(1)', 'url': 'https://www.amazon.in/Nothing-Phone-Black-128-RAM/dp/B0BKZVF7VV', 'alertPrice': 27950.0, 'price': 26999.0, 'alert': True}, {'product': 'iPhone 14', 'url': 'https://www.amazon.in/Apple-iPhone-128GB-Space-Black/dp/B0BDJ7P6NG', 'alertPrice': 121990.0, 'price': 122999.0, 'alert': False}, {'product': 'Nike Court Shoes', 'url': 'https://www.amazon.in/Nike-Legacy-Sail-Vivid-Green-Black-Running/dp/B0B291MD54', 'alertPrice': 3200.0, 'price': 3124.0, 'alert': True}]
running
mailed
